# Quantitative Momentum Strategy

In [3]:
#Imports
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats

from decouple import config

In [4]:
FMP_API_KEY = config('FMP_API_KEY')
IEX_API_KEY = config('IEX_API_KEY')
stocks = pd.read_csv('./data/sp_500_stocks.csv')

In [5]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_API_KEY}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2221511879150,
 'week52high': 136.69,
 'week52low': 55.74,
 'week52change': 0.7167588425106153,
 'sharesOutstanding': 16823263000,
 'float': 0,
 'avg10Volume': 122158231,
 'avg30Volume': 117078262,
 'day200MovingAvg': 114.55,
 'day50MovingAvg': 125.97,
 'employees': 0,
 'ttmEPS': 3.28,
 'ttmDividendRate': 0.8052302508604917,
 'dividendYield': 0.0060979193552479485,
 'nextDividendDate': '',
 'exDividendDate': '2020-11-06',
 'nextEarningsDate': '',
 'peRatio': 38.694882150633156,
 'beta': 1.1569139962790056,
 'maxChangePercent': 49.520315249827846,
 'year5ChangePercent': 4.879943182071184,
 'year2ChangePercent': 2.5160358393354016,
 'year1ChangePercent': 0.7167588425106153,
 'ytdChangePercent': -0.004823272288793312,
 'month6ChangePercent': 0.38152904558888934,
 'month3ChangePercent': 0.130870054852122,
 'month1ChangePercent': 0.07148653034729002,
 'day30ChangePercent': 0.07875173596928375,
 'day5ChangePercent': -0.004823272288793312}

## Batch API Call & Final DataFrame

In [6]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_API_KEY}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,127.695,0.462389,N/A
1,AAL,14.925,-0.444146,N/A
2,AAP,171.440,0.150171,N/A
3,AAPL,129.150,0.716759,N/A
4,ABBV,108.980,0.271148,N/A
...,...,...,...,...
500,YUM,107.370,0.073968,N/A
501,ZBH,158.100,0.0681851,N/A
502,ZBRA,411.210,0.646445,N/A
503,ZION,49.805,-0.00214236,N/A


## Removing Low Momentum Stocks

In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,41.635,2.39083,N/A
1,ALB,177.430,1.59655,N/A
2,LB,46.930,1.43829,N/A
3,FCX,30.390,1.42369,N/A
4,NVDA,551.445,1.17725,N/A
5,PYPL,238.600,1.14699,N/A
6,WST,302.090,1.02119,N/A
7,AMD,97.850,0.963667,N/A
8,ALGN,554.750,0.945011,N/A
9,CDNS,134.920,0.86485,N/A


## Calculating Shares to Buy

In [24]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio: 100000


100000


In [26]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,37.5400,2.12833,52
1,NVDA,518.8900,1.42895,3
2,LB,41.0200,1.32901,47
3,AMD,91.6600,1.32404,21
4,ALB,141.4300,1.21485,13
5,PYPL,215.3400,1.07817,9
6,FCX,24.8600,1.03194,78
7,NOW,538.0000,1.01664,3
8,QCOM,155.7500,0.899041,12
9,FDX,290.2500,0.876469,6


## Filtering Out Low-Quality Momentum

In [27]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_API_KEY}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

In [68]:
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

for row in hqm_dataframe.index:
    for time_period in time_periods:
        #print(hqm_dataframe[f'{time_period} Price Return'])
        print(hqm_dataframe.loc[row, f'{time_period} Price Return'])
        #stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
        #hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0.4449552442109359
0.3280721659279795
0.2075095078288407
0.07813067150635211
-0.33124168234165785
0.056991774383078564
0.3833141099577084
0.4531502423263325
0.019836601833797873
0.13092238578734627
0.03201516909668345
0.018015699395187212
0.8519715141414945
0.4020541617936826
0.08778449086231022
0.06268862636888839
0.310542005254848
0.1381178966895158
0.215324691476672
0.08718519267725289
0.16574673134564732
0.033454944817229215
0.058489056669695394
-0.09353377697300302
0.49057023643949926
0.0680884991030497
-0.01503565914270999
-0.0033849129593809257
0.2795579446830234
0.1641708179141539
0.027644723576882857
-0.038714917426225126
0.2341414379432314
0.1843064955958964
0.057667271350777316
0.03120941728457116
0.5677745997304138
0.17218917457352156
0.0012808130013017838
0.05502212389380534
0.2603190405589364
0.16080158410635192
0.24184402586706266
0.05286770070476843
0.15603133502821964
0.2085363936487974
0.07682913216764398
-0.011851753661872522
0.051757474510168544
0.12215066699461308


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.81,N/A,0.444955,N/A,0.328072,N/A,0.20751,N/A,0.0781307,N/A,N/A
1,AAL,17.99,N/A,-0.331242,N/A,0.0569918,N/A,0.383314,N/A,0.45315,N/A,N/A
2,AAP,158.22,N/A,0.0198366,N/A,0.130922,N/A,0.0320152,N/A,0.0180157,N/A,N/A
3,AAPL,123.24,N/A,0.851972,N/A,0.402054,N/A,0.0877845,N/A,0.0626886,N/A,N/A
4,ABBV,107.49,N/A,0.310542,N/A,0.138118,N/A,0.215325,N/A,0.0871852,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.46,N/A,0.0933945,N/A,0.142117,N/A,0.157972,N/A,0.0383404,N/A,N/A
501,ZBH,145.99,N/A,0.0132256,N/A,0.0913141,N/A,0.0546444,N/A,-0.0277704,N/A,N/A
502,ZBRA,375.14,N/A,0.460769,N/A,0.404598,N/A,0.503929,N/A,0.0872993,N/A,N/A
503,ZION,42.64,N/A,-0.113372,N/A,0.181357,N/A,0.382051,N/A,0.0938943,N/A,N/A
